# Прогнозирование временных рядов

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge, TheilSenRegressor, HuberRegressor
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, MinMaxScaler, RobustScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer

from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
RANDOM_STATE=177013

In [ ]:
df = pd.read_csv('PJME_hourly.csv')

In [ ]:
df.head()

## Приведение к правильному формату

In [ ]:
df = pd.read_csv('PJME_hourly.csv', index_col=['Datetime'], parse_dates=['Datetime'])

In [ ]:
df.head()

Также следует проверить, идут ли даты по порядку:

In [ ]:
df.index.is_monotonic_increasing

И при необходимости отсортировать данные:

In [ ]:
df = df.sort_index()

In [ ]:
plt.plot(df);

## Ресемплинг

In [ ]:
df.resample('1M').sum()

In [ ]:
plt.plot(df.resample('1m').mean());

In [ ]:
df_day = df['2016-01':'2018-01'].resample('1d').sum()

## Скользящие средние

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
plt.plot(df_day)
plt.plot(df_day.rolling(30).mean());
plt.plot(df_day.ewm(90).mean());

plt.xlabel('Время')
plt.ylabel('Расход за день, кВт-ч');

## Тренд и сезонность

**Тренд** - плавное изменение среднего без цикличности.

**Сезонность** - цикличные закономерности во временном ряду.

**Стационарным** рядом называется тот, у которого среднее, стандартное отклонение и ковариация со временем не меняются.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
decomposed = seasonal_decompose(df_day) 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
plt.title('Тренд и сезонность (1д)')
plt.plot(decomposed.trend);
plt.plot(decomposed.seasonal);
plt.xlabel('Время')
plt.ylabel('Расход за день, кВт-ч');

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
plt.title('Остатки (1д)')
plt.plot(decomposed.resid);
plt.xlabel('Время')
plt.ylabel('Расход за день, кВт-ч');

## Проверка на стационарность

Тест Дики-Фуллера: нулевая гипотеза - ряд нестационарен.

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
stat, p, *_ = adfuller(df_day['PJME_MW'])
print(f'P-value: {p:.2f}')
print(f'Результат: {stat:.2f}')
if p < 0.05:
    print ("Отвергаем нулевую гипотезу - ряд стационарный.")
else:
    print ("Не получилось отвергнуть нулевую гипотезу - ряд нестационарный.")

## Разности

In [ ]:
df_day.head()

In [ ]:
df_day.shift(fill_value=0).head()

In [ ]:
diff = df_day - df_day.shift(fill_value=df_day['PJME_MW'].iloc[0])

In [ ]:
decomposed = seasonal_decompose(diff)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12, 16))
ax[0].set(title = 'Тренд (первая разность, 1д)', xlabel='Время', ylabel='Расход за день, кВт-ч')
ax[0].plot(decomposed.trend);
ax[1].set(title = 'Сезонность (первая разность, 1д)', xlabel='Время', ylabel='Расход за день, кВт-ч')
ax[1].plot(decomposed.seasonal);
ax[2].set(title = 'Остатки (первая разность, 1д)', xlabel='Время', ylabel='Расход за день, кВт-ч')
ax[2].plot(decomposed.resid);

In [ ]:
stat, p, *_ = adfuller(diff['PJME_MW'])
print(f'P-value: {p}')
print(f'Результат: {stat:.2f}')
if p < 0.05:
    print ("Отвергаем нулевую гипотезу - ряд стационарный.")
else:
    print ("Не получилось отвергнуть нулевую гипотезу - ряд нестационарный.")

## Baseline

In [ ]:
data = df.resample('1d').sum()

In [ ]:
train, test = train_test_split(data, shuffle=False, test_size=0.2)

In [ ]:
print(f"Средний объём электропотребления в день: {test['PJME_MW'].mean():.2f} кВт-ч")

### На базе среднего/медианы

In [ ]:
print(f"MAE: {mean_absolute_error(test['PJME_MW'], np.ones(test.shape) * train['PJME_MW'].median()):.2f}")
print(f"MSE: {mean_squared_error(test['PJME_MW'], np.ones(test.shape) * train['PJME_MW'].median()):.2f}")

### На базе вчерашнего дня

In [ ]:
yesterday = test.shift(fill_value=0)
yesterday.iloc[0] = train.iloc[-1]

In [ ]:
print(f"MAE: {mean_absolute_error(test['PJME_MW'], yesterday):.2f}")
print(f"MSE: {mean_squared_error(test['PJME_MW'], yesterday):.2f}")

## Классический подход к решению

Классические методы включают в модель подмножество следующих компонентов:

- авторегрессия (AR): линейная регрессия следующей точки по нескольким предыдущим;
- так называемый MA-процесс: регрессия ошибки по предыдущим ошибкам;
- сведение ряда к более стационарному с помощью разностей и/или логарифмирования.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
model = ARIMA(train, order=(7, 0, 7)).fit()

In [ ]:
fig, axes = plt.subplots(figsize=(12,4))
train[-365:].plot(ax=axes)
model.forecast(90).plot(ax=axes);
test[:90].plot(ax=axes)
axes.set(ylabel='Расход за день, кВт-ч', title='Прогноз с помощью ARIMA');
axes.legend(['Обучение', 'Ожидание', 'Реальность']);

## Графики автокорреляции

In [ ]:
from statsmodels.graphics.tsaplots import acf, pacf, plot_acf, plot_pacf

Предыдущие значения за пределами синей зоны информативны.

In [ ]:
plot_acf(df_day);

Предыдущие ошибки за пределами синей зоны информативны.

In [ ]:
plot_pacf(df_day, method='ywm');

## Оценка модели на базе информационного критерия

При сравнении моделей для временных рядов часто используется **критерий Акаике**:

$$
AIC = 2k - 2ln(L)
$$

Где k - число параметров модели, а L - функция правдоподобия (исходим из допущения, что ошибки модели распределены нормально). Чем эта метрика меньше, тем лучше.

In [ ]:
model.aic

## Модель Хольта-Уинтерса

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
model_es = ExponentialSmoothing(endog=train, trend='add', seasonal='add', seasonal_periods=365, damped_trend=True, use_boxcox=True)

In [ ]:
fit_es = model_es.fit(optimized=True, remove_bias=False)

In [ ]:
fig, axes = plt.subplots(figsize=(12,4))
train[-365:].plot(ax=axes)
fit_es.forecast(365).plot(ax=axes);
test[:365].plot(ax=axes)
axes.set(ylabel='Расход за день, кВт-ч', title='Прогноз с помощью модели Хольта-Уинтерса');
axes.legend(['Обучение', 'Ожидание', 'Реальность']);

In [ ]:
fit_es.aic

## Используем sklearn

### Feature engineering

In [ ]:
data['year'] = data.index.year
data['month'] = data.index.month
#data['day'] = data.index.day
data['dayofweek'] = data.index.dayofweek

In [ ]:
data.head()

In [ ]:
max_lag = 7

In [ ]:
for i in range(1, max_lag+1):
    data['lag_'+str(i)] = data['PJME_MW'].shift(i)

In [ ]:
data['MA7'] = data['PJME_MW'].shift().rolling(7).mean()

In [ ]:
data.head()

In [ ]:
data = data.dropna()

### Моделирование

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('PJME_MW', axis=1), data['PJME_MW'], shuffle=False, test_size=0.2)

In [ ]:
model = LinearRegression()

model.fit(X_train, y_train)

print(f"MAE: {mean_absolute_error(y_test, model.predict(X_test)):.2f}")
print(f"MSE: {mean_squared_error(y_test, model.predict(X_test)):.2f}")
print(f"R2: {r2_score(y_test, model.predict(X_test)):.2f}")

In [ ]:
predictions = pd.Series(model.predict(X_test), index=y_test.index)

In [ ]:
fig, axes = plt.subplots(figsize=(12,4))
y_train[-90:].plot(ax=axes)
predictions[:90].plot(ax=axes)
y_test[:90].plot(ax=axes)
axes.legend(['Исторические данные', 'Ожидание', 'Реальность'])
axes.set(ylabel='Расход за день, кВт-ч', title='Прогноз с помощью sklearn');

### Оценим AIC для нашей модели!

In [ ]:
# Стандартизируем ошибки:
errors = y_test - predictions
normalized_errors = StandardScaler().fit_transform(errors.values.reshape(-1, 1))

# Сложим логарифмы их вероятностей на стандартном нормальном распределении:
log_likelihood = np.sum(st.norm.logpdf(normalized_errors))

num_params = model.coef_.size + 1
2 * num_params - 2 * log_likelihood

### Кросс-валидация для временных рядов

Важно, чтобы модель обучалась на прошлом и предсказывала будущее, а не наоборот. Чтобы гарантировать это при кросс-валидации, вам понадобится `TimeSeriesSplit()` из `sklearn.model_selection`. При подборе гиперпараметров нужно указать соответствующее разбиение,  например, так:

In [ ]:
params = {}

tscv = TimeSeriesSplit(n_splits=8)
gcv = GridSearchCV (estimator=model, param_grid=params, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Домашнее задание

Вам предстоит работать с данными по продажам 4 продуктов в 6 странах по 2 магазинам.

**Easy**: постройте модель sklearn для предсказания продаж одного из продуктов в одной стране за последний год по предыдущим данным.

**Medium**: проведите исследовательский анализ, распишите найденные изменения методологии, которые могут осложнить прогнозирование. Обработайте данные, извлеките признаки и постройте соответствующую модель (или отдельные модели) для пронозирования продаж по продуктам, странам и магазинам.

**Hard**: мы исходили из предположения, что нам доступны данные вплоть до текущего дня. Предположим, что у нас есть большая закрытая тестовая выборка - для нее мы не сможем построить отстающие значения. Попробуйте извлечь признаки из исторических данных, которые не потребуют знания продаж за последние дни, и построить соответствующую модель.

In [ ]:
df = pd.read_csv('kaggle_sales.csv')

In [ ]:
df.head()

Ввиду небольшого разброса рекомендуется оценивать модели по метрике SMAPE:

$$
SMAPE = \frac{1}{n} \sum_n {\frac{2|y-\hat y|}{|y|+|\hat y|}} \cdot 100 \%
$$

In [ ]:
def smape(target, predictions):
    result = np.mean(np.nan_to_num(abs(target - predictions) / ((abs(target)+abs(predictions))/2), posinf=0)) * 100.
    return result

In [ ]:
# Скорер для кросс-валидации:
smape_score = make_scorer(smape, greater_is_better=False)

Пример тестовой выборки для Hard:

In [ ]:
df_test = pd.read_csv('kaggle_sales_test.csv')

In [ ]:
df_test